In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE,RandomOverSampler
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
import dtale

In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

df=pd.read_csv(r'C:\Users\Pradeep Sharma\Downloads\Case_Study_Data_Test.csv')
df.dropna(axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['Sample_label_encoding']=label_encoder.fit_transform(df['Sample'])
df['Package_label_encoding']=label_encoder.fit_transform(df['Package'])
def preprocess(tweet):
   
   # Removing handles and hashtags
   tweet = re.sub('@[^\s]+','',str(tweet))
   tweet = re.sub(r'#', '', str(tweet))
   # Removing URLS
   tweet= re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',str(tweet))
   #removing all punchuation and special character
   tweet = re.sub('[^a-zA-Z]',' ',str(tweet))
   #removing extra white space
   tweet = re.sub('[\s]+', ' ', str(tweet))
   tweet = re.sub('[\n]+', ' ', str(tweet))
   tweet=tweet.lower()
   return tweet
train_data= df['Medical_Description'].apply(lambda x:preprocess(x))
type(train_data)

train_data=pd.DataFrame(train_data)
train_data.columns=['Medical_Description']
train_data.reset_index(drop=True,inplace=True)

def token(tweet):
  tweet = word_tokenize(tweet)
  return tweet
train_data['Medical_Description']=train_data['Medical_Description'].apply(lambda x:token(x))

In [3]:
#Performing Text Normalization
lem = WordNetLemmatizer()
def normalize(tweet):
  tweet = [lem.lemmatize(word) for word in tweet if not word in stop_words ]
  #print(tweet)
  tweet = ' '.join(tweet)
  return tweet
train_data['Medical_Description']= train_data['Medical_Description'].apply(lambda x :normalize(x))
train_data.dropna(inplace=True)
x_tf=train_data['Medical_Description']
y_tf=df['Package_label_encoding']
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format(r'D:\nlp\GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)
tfidf = TfidfVectorizer()
tfidf.fit(x_tf)
dictionary = dict(zip(tfidf.get_feature_names(), list(tfidf.idf_)))
def word2vectransform(list_of_sentance):
    zero = np.zeros(300)
    tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
    for sent in list_of_sentance: # for each review/sentence 
        sent = sent.split()
        sent_vec = np.zeros(300) # as word vectors are of zero length
        weight_sum =0; # num of words with a valid vector in the sentence/review
        for word in sent: # for each word in a review/sentence
            try:
                vec = model[word]
            except:
                vec = zero
            try:
                tf_idf = dictionary[word]*(sent.count(word)/len(sent))
            except:
                tf_idf = 0
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        if weight_sum != 0:
                sent_vec /= weight_sum
        tfidf_sent_vectors.append(sent_vec)
        
    return tfidf_sent_vectors
df1 = word2vectransform(x_tf)
df1=pd.DataFrame(df1)
df_final=pd.concat([df1,df['Sample_label_encoding']],axis=1)

In [30]:
import joblib
log=joblib.load(r'C:\Users\Pradeep Sharma\logistic_med_with_out_sample_col.pkl')
cc=log.predict(df_final)

In [31]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_tf, cc))

,0,1,2,3,4,5,6,7,8,9,10,11
0,30,0,2,1,2,0,1,0,0,0,1,0
1,0,9,0,0,0,0,0,0,0,0,1,0
2,0,1,19,0,2,0,0,0,0,1,0,0
3,1,0,1,3,2,2,0,0,0,0,0,0
4,0,0,1,0,6,0,0,0,0,0,1,0
5,1,0,1,2,2,10,1,0,0,5,0,0
6,0,0,0,2,0,0,4,0,0,4,0,0
7,0,0,0,1,0,0,0,15,0,0,0,0
8,0,0,0,0,0,0,0,0,8,0,0,0
9,1,0,1,0,0,3,2,0,0,29,0,0


In [32]:
print(classification_report(y_tf,cc))

              precision    recall  f1-score   support

           0       0.88      0.81      0.85        37
           1       0.90      0.90      0.90        10
           2       0.73      0.83      0.78        23
           3       0.27      0.33      0.30         9
           4       0.38      0.75      0.50         8
           5       0.62      0.45      0.53        22
           6       0.50      0.40      0.44        10
           7       1.00      0.94      0.97        16
           8       1.00      1.00      1.00         8
           9       0.72      0.81      0.76        36
          10       0.50      0.43      0.46         7
          11       0.73      0.53      0.62        15

    accuracy                           0.72       201
   macro avg       0.69      0.68      0.67       201
weighted avg       0.73      0.72      0.72       201



In [33]:
cc=pd.DataFrame(cc)
cc.columns=['predictions']

In [34]:
cc

,predictions
0,8
1,9
2,0
3,8
4,6
...,...
196,8
197,6
198,5
199,4


In [35]:
df_new=pd.concat([df,cc],axis=1)

In [26]:
df_new.to_csv('output.csv')